In [0]:
# -------------------------------------------------------------------------
# UI SETUP: FARMER MODE ONLY
# -------------------------------------------------------------------------
import mlflow
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, current_date, lit
from IPython.display import display, HTML

# 1. Clean up old widgets (Resetting the toolbar)
dbutils.widgets.removeAll()

# 2. Create Farmer-Specific Inputs
# Region Selector
states_list = [row['state'] for row in spark.sql("SELECT DISTINCT state FROM agriculture.gold.roi_recommendations ORDER BY state").collect()]
dbutils.widgets.dropdown("selected_state", "Punjab", states_list, "1. Select Region")

# Soil Inputs (Default values set to fertile soil)
dbutils.widgets.text("input_n", "90", "2. Nitrogen (N)")
dbutils.widgets.text("input_p", "42", "3. Phosphorus (P)")
dbutils.widgets.text("input_k", "43", "4. Potassium (K)")
dbutils.widgets.text("input_ph", "6.5", "5. Soil pH")

# 3. Display App Header
display(HTML("""
<div style="background-color:#003c2f; padding:20px; border-radius:10px; border-left: 10px solid #2ecc71;">
    <h1 style="color:white; margin:0; font-family:sans-serif;"> AgriFuture: Farmer Edition</h1>
    <h3 style="color:#a5d6a7; margin:0; font-family:sans-serif;">Precision Farming & Profitability Engine</h3>
    <p style="color:white; margin-top:5px;">
       Enter your soil details above to get AI-driven crop recommendations for the <b>2026 Harvest Season</b>.
    </p>
</div>
"""))


# -------------------------------------------------------------------------
# 1. GET USER INPUTS
# -------------------------------------------------------------------------
region = dbutils.widgets.get("selected_state")
n = float(dbutils.widgets.get("input_n"))
p = float(dbutils.widgets.get("input_p"))
k = float(dbutils.widgets.get("input_k"))
ph = float(dbutils.widgets.get("input_ph"))

# -------------------------------------------------------------------------
# 2. AUTO-ENRICHMENT (Fetch Silver Layer Climate Data)
# -------------------------------------------------------------------------
# We look up the typical weather for this state to help the model
try:
    weather_row = spark.table("agriculture.silver.weather_history") \
        .filter(col("state_name") == region) \
        .agg({"avg_temperature": "avg", "avg_humidity": "avg", "total_rainfall": "avg"}) \
        .collect()[0]
    
    avg_temp = weather_row[0]
    avg_hum = weather_row[1]
    avg_rain = weather_row[2]
except:
    # Fallback if state not found in weather history
    avg_temp, avg_hum, avg_rain = 25.0, 70.0, 100.0

# -------------------------------------------------------------------------
# 3. AI MODEL: THE AGRONOMIST (Will it grow?)
# -------------------------------------------------------------------------
# Load the registered model
model_uri = "models:/agriculture.silver.crop_classifier/1" # Check your model name in registry if this fails!
model = mlflow.sklearn.load_model(model_uri)

# Create Input DataFrame (Order must match training!)
input_data = pd.DataFrame([[n, p, k, avg_temp, avg_hum, ph, avg_rain]], 
                          columns=['nitrogen', 'phosphorus', 'potassium', 'temperature', 'humidity', 'ph', 'rainfall'])

# Predict Top 3 Crops
probs = model.predict_proba(input_data)[0]
top3_indices = np.argsort(probs)[-3:][::-1]
top3_crops = model.classes_[top3_indices]

# -------------------------------------------------------------------------
# 4. MARKET CHECK: THE ECONOMIST (Will it make money?)
# -------------------------------------------------------------------------
# Query the Gold Table for these specific crops in 2026
rec_df = spark.table("agriculture.gold.roi_recommendations") \
    .filter(col("state") == region) \
    .filter(col("crop_name").isin(list(top3_crops))) \
    .filter(col("forecast_date") >= current_date()) \
    .orderBy(col("estimated_revenue").desc()) \
    .limit(3) \
    .select("crop_name", "forecast_date", "predicted_price", "estimated_revenue", "risk_category")

# -------------------------------------------------------------------------
# 5. RENDER UI 
# -------------------------------------------------------------------------
display(HTML(f"""
<div style="border: 1px solid #ddd; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
    <h3 style="margin-top:0;"> Analysis for {region}</h3>
    <p><b>Detected Climate:</b> Temp: {avg_temp:.1f}°C | Rain: {avg_rain:.1f}mm | Soil pH: {ph}</p>
    <p><b>AI Agronomist Assessment:</b> Based on your soil (N={n}, P={p}, K={k}), the biologically viable crops are: 
       <span style="background-color:#e8f5e9; color:#2e7d32; padding:2px 8px; border-radius:4px;"><b>{', '.join(top3_crops).upper()}</b></span>
    </p>
</div>
"""))

print("DETAILED 2026 PROFITABILITY FORECAST:")
display(rec_df)  

if rec_df.count() > 0:
    best_crop = rec_df.first()['crop_name']
    revenue = rec_df.first()['estimated_revenue']
    
    display(HTML(f"""
    <div style="background-color:#fff3e0; padding:15px; border-left: 5px solid #ff9800; border-radius:5px; margin-top:10px;">
        <h3 style="margin:0; color:#e65100;">Final Recommendation: {best_crop.upper()}</h3>
        <p style="margin:5px 0 0 0;">
           Projected Revenue: <b>₹{revenue:,.2f} / acre</b><br>
           Why? It balances biological suitability with high predicted market demand in 2026.
        </p>
    </div>
    """))
else:
    display(HTML("<b style='color:red'>No profitable crops found for 2026 in this region.</b>"))

DETAILED 2026 PROFITABILITY FORECAST:


DataFrame[crop_name: string, forecast_date: date, predicted_price: double, estimated_revenue: double, risk_category: string]